# Table selection

In [6]:
from utils.table_selection.table_selector import *
from utils.table_selection.table_details import *

In [2]:
table_descriptions = get_table_schemas()
print(table_descriptions, "\n")

table name: Consumer Price Index - Products
table description: Table 'Consumer Price Index - CPI' has price index for products, encompassing over 200 expenditure categories grouped into major segments (food and beverages, housing, apparel, recreation, and other goods).
table columns: Year [INTEGER] (year), Consumer Price Inde [FLOAT] (consumer price index), Standard Error [FLOAT] (standard error or CPI), Percent Change [FLOAT] (percentage change), Product or Service [TEXT] (enums `PRODUCTS_1`)


table name: dot_faf
table description: Table 'dot_faf' has freight movement among states and major metropolitan areas by all modes of transportation. Shows which goods are shipped from one region of the US to another region, according to type of commodity, mode of shipment, value, and weight.
table columns: Year [INTEGER] (year), Millions Of Dollars [FLOAT] (value in millions of dollars of a certain shipment.), Thousands Of Tons [FLOAT] (weight in thousands of tons of a certain shipment.), Orig

In [3]:
get_table_names()

['Consumer Price Index - Products',
 'dot_faf',
 'us_location_data',
 'ed_defaults',
 'Consumer Price Index - Services',
 'pums_5']

In [4]:
query = 'What were the average salaries of managers in California during 2015?'

In [5]:
tables = get_relevant_tables_from_lm(query)


Messages:
 [{'role': 'user', 'content': 'What service presented the highest growth in the consumer price index during the last year?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["Consumer Price Index - Services"]\n}\n'}, {'role': 'user', 'content': 'What is the average monthly CPI volatility for all product groups combined?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["Consumer Price Index - Products"]\n}\n'}, {'role': 'user', 'content': 'Which states or metropolitan areas are the primary origins for specific products, and which are the primary destinations?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["Consumer Price Index - Products"]\n}\n'}, {'role': 'user', 'content': 'What is the trend in total freight movement volume across all transportation modes over the past five years?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["dot_faf", "us_location_data"]\n}\n'}, {'role': 'user', 'content': 'What is the trend in total freight movement volu

In [6]:
print(tables)


['pums_5']


In [7]:
tables_db = get_relevant_tables_from_database(query)

DF:   table_name  similarity
0     pums_5    0.344073


In [8]:
print(tables_db)

['pums_5']


In [9]:
table_details = {}
with open("data/tables.json", "r") as f:
    table_details = json.load(f)

In [12]:
table_ = 'pums_5'

In [15]:
get_table_schemas(table_)

"table name: pums_5\ntable description: Table 'pums_5' has data on total population and average wages by Year, Nation, State or PUMA, age, gender, race, PUMS occupation and PUMS industry. You can query any combination of these.\ntable columns: Year [INTEGER] (year), Nation [TEXT], State [TEXT], PUMA [TEXT] (public use microdata areas), Age [INTEGER], Gender [TEXT], Race [TEXT], PUMS occupation [TEXT] (PUMS Occupation classification), PUMS industry [TEXT] (PUMS industry classification)\n"

In [10]:
table_selected = request_tables_to_lm_from_db(query)

DF:                         table_name  similarity
0                           pums_5    0.344073
1  Consumer Price Index - Products    0.101050
2                          dot_faf    0.064463
Messages:
 [{'role': 'user', 'content': 'What service presented the highest growth in the consumer price index during the last year?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["Consumer Price Index - Services"]\n}\n'}, {'role': 'user', 'content': 'What is the average monthly CPI volatility for all product groups combined?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["Consumer Price Index - Products"]\n}\n'}, {'role': 'user', 'content': 'Which states or metropolitan areas are the primary origins for specific products, and which are the primary destinations?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["Consumer Price Index - Products"]\n}\n'}, {'role': 'user', 'content': 'What is the trend in total freight movement volume across all transportation modes over th

In [16]:
table_selected

['pums_5']

# API parameter generator

In [1]:
from utils.api_data_request.api_generator import *

In [2]:
v, m, c = get_api_params_from_lm('how much did the average salaries of women grow between 2015 and 2018?', table_list='pums_5')


ChatGPT response: The chosen variables, measures and filters can answer the query because they directly relate to the information requested: the growth of average salaries of women between 2015 and 2018. The variables are time-related (Year), the measure is a numeric value (Average Wage), and the filter is based on a specific demographic group (Gender = Women).

- Variables: "Year" is a column in the table and it is an integer, which allows us to filter the data for the years 2015 and 2018.
- Measures: "Average Wage" is a column in the table and it is an integer, which allows us to calculate the average salaries.
- Filters: "Gender" is a column in the table and it is a text, which allows us to filter the data for women.

The JSON object would look like this:

```
{
    "variables": ["Year"],
    "measures": ["Average Wage"],
    "filters": ["Gender = Women", "Year >= 2015", "Year <= 2018"]
}
```

Parameters: 
{
    "variables": ["Year"],
    "measures": ["Average Wage"],
    "filters"

In [3]:
print("Variables:\n\n", v, "\n", "\nMeasures:\n\n", m, "\n", "\nCuts:\n\n", c)

Variables:

 ['Year'] 
 
Measures:

 ['Average Wage'] 
 
Cuts:

 ['Gender = Women', 'Year >= 2015', 'Year <= 2018']


In [4]:
api_url = api_build(drilldown = v[0].strip("'\""), measure = m[0])

In [5]:
api_url

'https://zircon.datausa.io/api/data?drilldowns=Year&measures=Average+Wage'

# API Request

In [8]:
data, df = api_request(api_url)

In [9]:
data

[{'ID Year': 2021, 'Year': '2021', 'Average Wage': 27936.525448568376},
 {'ID Year': 2020, 'Year': '2020', 'Average Wage': 26200.58132413172},
 {'ID Year': 2019, 'Year': '2019', 'Average Wage': 25356.424307177695},
 {'ID Year': 2018, 'Year': '2018', 'Average Wage': 24316.61975751166},
 {'ID Year': 2017, 'Year': '2017', 'Average Wage': 23247.646200745152},
 {'ID Year': 2016, 'Year': '2016', 'Average Wage': 22225.811091529966},
 {'ID Year': 2015, 'Year': '2015', 'Average Wage': 21547.164278533583},
 {'ID Year': 2014, 'Year': '2014', 'Average Wage': 21282.953902287813}]

In [10]:
df

,ID Year,Year,Average Wage
0,2021,2021,27936.525449
1,2020,2020,26200.581324
2,2019,2019,25356.424307
3,2018,2018,24316.619758
4,2017,2017,23247.646201
5,2016,2016,22225.811092
6,2015,2015,21547.164279
7,2014,2014,21282.953902
